In [1]:
from Linear import *
from ReLU import *
from LossMSE import *
from Tanh import *
from Sequential import *

from torch import FloatTensor
from torch import LongTensor 
import math

In [2]:
def convert_to_one_hot_labels(target):
    tmp = FloatTensor(target.size(0), target.max() + 1).fill_(-1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [3]:
#data generation
def generate_disc_set(nb):
    uniform_input = FloatTensor(nb, 2).uniform_(0, 1)
    target = uniform_input.sub(0.5).pow(2).sum(1).sub(1 /(2*math.pi)).sign().mul(-1).add(1).div(2).long()
    print(target.sum())
    return uniform_input, convert_to_one_hot_labels(target)

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)
mini_batch_size = 100

506
495


In [4]:
#data noramlization
mean, std = train_input.mean(0), train_input.std(0)

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)


 1.2482  0.1433
 0.5294 -0.8195
-1.1946 -0.2279
       ⋮        
 1.4008  0.6319
-1.5091  1.6382
-0.2745  0.1937
[torch.FloatTensor of size 1000x2]

In [5]:
#create model
model = Sequential(
        Linear(2, 25), 
        Tanh(),
        Linear(25, 25), 
        Tanh(),
        Linear(25, 25), 
        Tanh(),
        Linear(25, 2)
)

In [6]:
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch

In [7]:
model_torch = nn.Sequential(
        nn.Linear(2, 25), 
        nn.Tanh(),
        nn.Linear(25, 25), 
        nn.Tanh(),
        nn.Linear(25, 25), 
        nn.Tanh(),
        nn.Linear(25, 2)
)

In [8]:
def train_model_torch(model, train_input, train_target):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1) 
    nb_epochs = 250

    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
train_model_torch(model_torch, Variable(train_input), Variable(train_target))

In [9]:
def compute_nb_errors_torch(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k, predicted_classes[k]] < 0:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors
print('train_error {:.02f}% test_error {:.02f}%'.format(
            compute_nb_errors_torch(model_torch, Variable(train_input), Variable(train_target)) / train_input.size(0) * 100,
            compute_nb_errors_torch(model_torch, Variable(test_input), Variable(test_target)) / test_input.size(0) * 100
        )
        )

train_error 0.70% test_error 1.50%


In [10]:
#train network
def train_model(model, train_input, train_target):
    criterion = LossMSE()
    nb_epochs = 250
    lr = 1e-1

    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            loss = criterion.forward(output, train_target.narrow(0, b, mini_batch_size))
            #print(loss)
            dl_din = criterion.backward()
            model.backward(dl_din)
            for p in model.param():
                p[0].sub_(lr*p[1])
                #if b<10: print(p[1])
train_model(model, train_input, train_target)

In [11]:
def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model.forward(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(0, mini_batch_size):
            #print(data_target.size())
            #print(type(predicted_classes.size())
            if data_target[b + k, predicted_classes[k]] < 0:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [12]:
print('train_error {:.02f}% test_error {:.02f}%'.format(
            compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
            compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100
        )
        )

train_error 0.50% test_error 1.20%
